In [19]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import yfinance as yf #stock api


In [20]:
#Tickers
start = '2020-01-02'
end = '2021-07-02'

ticker_adjclose = yf.download('MSFT, CAT, AAPL, AMD', period = "ytd", start = start, group_by = 'column',auto_adjust=True, threads = True)['Close']

[*********************100%***********************]  4 of 4 completed


In [21]:
#Given data
ret_exp = 0.006 #expected return
log_returns = np.log(1+ticker_adjclose.pct_change().dropna())
cov = log_returns.cov() #Covariance matrix
weight = np.full((len(cov),1), 1/len(cov)) #Start from equal weight portfolio
bounds = ((0,1),)*len(cov)

In [22]:
#Function and constrains
def risk_func(weight):
    return np.dot(weight.T, np.dot(cov,weight))

def Sum_to_one(weight):
    return np.sum(weight)-1

def Min_return(weight):
    return ret_exp - np.sum(log_returns.mean() * weight)

constraints = ({'type': 'eq', 'fun': Sum_to_one},{'type': 'eq', 'fun': Min_return})

In [23]:
#optimize function subject to constraints

#min_var_solve = minimize(risk_func, weight, method = 'SLSQP', bounds = bounds, constraints = constraints[0])
#optimizer_b = minimize(risk_func, weight, method = 'SLSQP', bounds = bounds, constraints = constraints)
optimizer = minimize(risk_func, weight, method = 'SLSQP', constraints = constraints)
weight_optim = optimizer.x
weight_optim

array([ 2.82559829,  1.77814838, -5.45873818,  1.8549915 ])

In [24]:
#check constraints just in case
[risk_func(weight_optim), Sum_to_one(weight_optim), Min_return(weight_optim)]

[0.023958695426704646, 4.440892098500626e-16, -3.016978854242236e-11]

In [25]:
#Solving by hand
ret_mean = log_returns.mean()

def variance_minimizer(ret_exp, ret_mean, cov):
    w_con = np.ones(len(cov)) #weight constraints
    cov_inverse = np.linalg.inv(cov) #This is what makes the code slow.

    #Matrix multiplications (so the code looks cleaner)
    cov_cons_mean = np.dot(ret_mean.T, np.dot(cov_inverse, ret_mean))
    cov_cons_weight = np.dot(w_con.T, np.dot(cov_inverse, w_con))
    cov_cons_wm = np.dot(ret_mean.T, np.dot(cov_inverse, w_con))

    return (1/(cov_cons_mean * cov_cons_weight - cov_cons_wm**2)) * np.dot(cov_inverse,((cov_cons_weight*ret_exp - cov_cons_wm) * ret_mean + (cov_cons_mean - cov_cons_wm*ret_exp) * w_con))



In [26]:
variance_minimizer(ret_exp, ret_mean, cov)

array([ 2.85292094,  1.77970676, -5.45520104,  1.82257334])